In [ ]:
!pip install sympy

In [85]:
import sympy as sp
import random
from sympy.core.basic import Basic
from pprint import pprint as pp
# import all operations from sympy
#from sympy import *

alphabet = {
    # symbols
    'q': sp.Symbol('q'),
    'r': sp.Symbol('r'),
    's': sp.Symbol('s'),
    't': sp.Symbol('t'),
    'u': sp.Symbol('u'),
    'v': sp.Symbol('v'),
    'w': sp.Symbol('w'),
    'x': sp.Symbol('x'),
    'y': sp.Symbol('y'),
    'z': sp.Symbol('z'),

    # wildcards
    'a': sp.Wild('a'),
    'b': sp.Wild('b'),
    'c': sp.Wild('c'),
    'd': sp.Wild('d'),
    'e': sp.Wild('e'),
    'f': sp.Wild('f'),
    'g': sp.Wild('g'),
    'h': sp.Wild('h'),

    # integers
    'i': sp.Wild('i', properties=[lambda x: x.is_integer and x > 0]),
    'j': sp.Wild('j', properties=[lambda x: x.is_integer and x > 0]),
    'k': sp.Wild('k', properties=[lambda x: x.is_integer and x > 0]),
    'l': sp.Wild('l', properties=[lambda x: x.is_integer]),
    'n': sp.Wild('n', properties=[lambda x: x.is_integer]),
    'm': sp.Wild('m', properties=[lambda x: x.is_integer]),
    'o': sp.Wild('o', properties=[lambda x: x.is_integer]),
    'p': sp.Wild('p', properties=[lambda x: x.is_integer])
}
for key, value in alphabet.items():
    globals()[key] = value

def pe(expr_str):
    return sp.parse_expr(
        expr_str,
        local_dict=alphabet, 
        evaluate=0)



In [67]:
expr1 = sp.Integer(3) * (x + 1) + sp.Pow(sp.sin(x), 2) + sp.Pow(sp.cos(x), 2)
expr1_simp = sp.simplify(expr1)
expr2 = pe('3 * (x+1) + sin(x)**2 + cos(x)**2')
expr2_simple = sp.simplify(expr2)
print(f"Default: {expr1}")
print(f"Unevaluated: {expr2}")

print(f"Simplify Default: {expr1_simp}")
print(f"Simplify Unevaluated: {expr2_simple}")


Default: 3*x + sin(x)**2 + cos(x)**2 + 3
Unevaluated: 3*(x + 1) + sin(x)**2 + cos(x)**2
Simplify Default: 3*x + 4
Simplify Unevaluated: 3*x + 4


In [88]:
def explore_expression(expr, depth=0):
    indent = "  " * depth
    print(f"{indent}{expr.__class__.__name__}: {expr}")
    
    if isinstance(expr, sp.Atom):  # Base case: atoms (symbols, numbers) have no args
        return
    
    for arg in expr.args:
        explore_expression(arg, depth + 1)

expr3 = 3 * (x**2 + 2*y) * sp.sin(4*z) + 5 * sp.sqrt(x*y)
expr3_2 = pe('5 + (3 * 2 / x)')

#print("Expression structure:")
#explore_expression(expr3)
print("Expression structure 2:")
explore_expression(expr3_2)

Expression structure 2:
Add: 5 + 3*2/x
  Integer: 5
  Mul: 3*2/x
    Integer: 3
    Integer: 2
    Pow: 1/x
      Symbol: x
      NegativeOne: -1


In [70]:
print(expr2.subs(x, 1.0))


7.00000000000000


In [76]:
def rewrite_mul_to_add(expr):
    if isinstance(expr, (sp.Number, sp.Symbol)):
        return expr

    # Handle multiplication
    if expr.func == sp.Mul:
        factors = list(expr.args)
        coeff = sp.Number(1)
        remaining_factors = []

        for factor in factors:
            if isinstance(factor, sp.Number):
                coeff *= factor
            else:
                remaining_factors.append(factor)

        if coeff.is_integer and coeff > 1:
            sub_expr = sp.Mul(*remaining_factors, evaluate=False)
            return sp.Add(*[rewrite_mul_to_add(sub_expr) for _ in range(int(coeff))], evaluate=False)

    # Recursively handle all arguments
    new_args = [rewrite_mul_to_add(arg) for arg in expr.args]
    return expr.func(*new_args, evaluate=False)

complex_expr = (
    3 * (x**2 + 2*y) * sp.sin(4*z) +
    5 * sp.sqrt(x*y) * (sp.cos(z)**2 + sp.sin(z)**2) +
    2 * (3*x + 4*y) * sp.exp(2*z) +
    7 * sp.log(x + y) * (x - y)**3
)


changed_expr = rewrite_mul_to_add(complex_expr)
#rewritten_expr3 = rewrite_mul_to_add(expr3.subs(n, 4))  # Substitute n with 4

print(f"{complex_expr} ---> {changed_expr}")


5*sqrt(x*y)*(sin(z)**2 + cos(z)**2) + 7*(x - y)**3*log(x + y) + (6*x + 8*y)*exp(2*z) + (3*x**2 + 6*y)*sin(4*z) ---> ((x**2 + x**2 + x**2) + (y + y + y + y + y + y))*sin(z + z + z + z) + ((x + x + x + x + x + x) + (y + y + y + y + y + y + y + y))*exp(z + z) + (sqrt(x*y)*(sin(z)**2 + cos(z)**2) + sqrt(x*y)*(sin(z)**2 + cos(z)**2) + sqrt(x*y)*(sin(z)**2 + cos(z)**2) + sqrt(x*y)*(sin(z)**2 + cos(z)**2) + sqrt(x*y)*(sin(z)**2 + cos(z)**2)) + ((x - y)**3*log(x + y) + (x - y)**3*log(x + y) + (x - y)**3*log(x + y) + (x - y)**3*log(x + y) + (x - y)**3*log(x + y) + (x - y)**3*log(x + y) + (x - y)**3*log(x + y))


In [89]:


# Define rules as tuples (pattern, replacement)
rules = [
    (sp.Add(a, -a, evaluate=False), 0),                             # a - a -> 0
    (a, sp.Add(a, sp.Integer(0), evaluate=False)),                  # a -> a + 0
    (a, sp.Mul(a, sp.Integer(1), evaluate=False)),                  # a -> a * 1
    (sp.Integer(0), sp.Mul(a, 0, evaluate=False)),                  # 0 -> a * 0
    (sp.Add(a, sp.Mul(-1, 0, evaluate=False), evaluate=False), a),  # a - 0 -> a
    (2 * a, sp.Add(a, a, evaluate=False)),                          # 2 * a -> a + a
    # n * a -> a + ... + a (n times)
    #(sp.Pow(a, 1, evaluate=False), a),
    #(a, sp.Pow(a, 1, evaluate=False)),
    #(1, sp.Pow(x, 0, evaluate=False)),
    #(1, sp.Pow(y, 0, evaluate=False)),
    #(1, sp.Pow(z, 0, evaluate=False)),
    #(sp.Pow(a, 1, evaluate=False), a),
    #(sp.Pow(a, 0, evaluate=False), sp.Integer(1)),
    #(1, sp.Pow(x, -x, evaluate=False)),
    #(1, sp.Pow(y, -y, evaluate=False)),
    #(1, sp.Pow(z, -z, evaluate=False)),
    #(sp.Pow(a, -a, evaluate=False), sp.Integer(1)),
    # do this: (a, b*a - (b-1)*a),
    #(a, sp.Add(sp.Mul(b, a, evaluate=False), sp.Mul(sp.Add(b, -1, evaluate=False), a, evaluate=False)),
#    (sp.sin(a) ** 2 + sp.cos(a) ** 2, 1),
    (1, sp.sin(x) ** 2 + sp.cos(x) ** 2),
    (1, sp.sin(y) ** 2 + sp.cos(y) ** 2),
    (1, sp.sin(z) ** 2 + sp.cos(z) ** 2),
#    ((a + b) * c, a * c + b * c),
#    ((a + b) * (a - b), a ** 2 - b ** 2),
#    ((a - b) * c, a * c - b * c),
]

pp(rules)

[(-a_ + a_, 0),
 (a_, a_ + 0),
 (a_, a_*1),
 (0, a_*0),
 (a_ - 1*0, a_),
 (2*a_, a_ + a_),
 (1, sin(x)**2 + cos(x)**2),
 (1, sin(y)**2 + cos(y)**2),
 (1, sin(z)**2 + cos(z)**2)]


In [91]:
# Function to apply a single rule
def apply_rule(expr, rule):
    pat, rep = rule
    return expr.replace(pat, rep, simultaneous=False, map=True)

# Function to apply rules randomly
def rewrite(expr, rules, steps=5, max_tries=1000):
    while steps > 0 and max_tries > 0:
        rule = random.choice(rules)
        #print(f'rule: {rule}')
        max_tries -= 1
        expr, map = apply_rule(expr, rule)
        #print(f'expr: {expr}, map: {map}')
        if map:
            print(f'[applying rule {rule[0]} -> {rule[1]}]')
            steps -= 1
    return expr


Original expression: 3*x
Rewritten expression: 3*x


In [92]:
from sympy import symbols

x, y, z = symbols('x y z')
a, b, c = sp.Wild('a'), sp.Wild('b'), sp.Wild('c'),

exploration_rules = [
    # Simplification rules
    (a + a, 2*a),             # Simplify:a + a -> 2a
    (a * 1, a),               # Simplify: a * 1 -> a
    (a + 0, a),               # Simplify: a + 0 -> a
    (a * (b + c), a * b + a * c),  # Distributive property
    (a * a, a**2),            # Simplify: a * a -> a^2
    (a * b, b * a),           # Commutative property of multiplication
    (1, a / a),               # Complexify: 1 -> a/a
    (a, a + 0),               # Complexify: a -> a + 0
    (a, a * 1),               # Complexify: a -> a * 1
    (a - a, 0),               # Simplify: a - a -> 0
    (a + b, b + a),           # Commutative property of addition
    (a - b, -b + a),          # Change subtraction order
    (a / a, 1),               # Simplify: a/a -> 1
    (a ** 1, a),              # Simplify: a^1 -> a
    (a ** 0, 1),              # Simplify: a^0 -> 1
    (a + b - b, a),           # Simplify: a + b - b -> a
    (a * b / b, a),           # Simplify: a * b / b -> a
    (a * (b - c), a * b - a * c),  # Distributive property
    (a / b * b, a),           # Simplify: a / b * b -> a
    (a - 0, a),               # Simplify: a - 0 -> a
    (a * 0, 0),               # Simplify: a * 0 -> 0
    (0 / a, 0),               # Simplify: 0 / a -> 0
    (a + (b + c), (a + b) + c),  # Associative property of addition
    (a * (b * c), (a * b) * c),  # Associative property of multiplication
    (a - (-b), a + b),        # Simplify: a - (-b) -> a + b
    ((a / b) / c, a / (b * c)),  # Simplify: (a / b) / c -> a / (b * c)
    (a * (-1), -a),           # Simplify: a * (-1) -> -a
    (-(a * b), (-a) * b),     # Simplify: -(a * b) -> (-a) * b
    (a + (b - a), b),         # Simplify: a + (b - a) -> b
    ((a + b) * c, a * c + b * c),  # Distributive property

    # Complexification rules
    (a, a + b - b),           # Complexify: a -> a + b - b
    (a, a * b / b),           # Complexify: a -> a * b / b
    (a, a * 1),               # Complexify: a -> a * 1
    (a, a / a * a),           # Complexify: a -> a / a * a
    (a, (a + b) - b),         # Complexify: a -> (a + b) - b
    (a, (a * b) / b),         # Complexify: a -> (a * b) / b
    (a, (a + 0)),             # Complexify: a -> a + 0
    (a, a * 1),               # Complexify: a -> a * 1
    (a, a / (1 / a)),         # Complexify: a -> a / (1 / a)
    (a, a * a / a),           # Complexify: a -> a * a / a
    (a, (a + b) + (c - c)),   # Complexify: a -> (a + b) + (c - c)
    (a, (a * b) * (1 / b)),   # Complexify: a -> (a * b) * (1 / b)
    (a, a + (b - b)),         # Complexify: a -> a + (b - b)
    (a, (a - b) + b),         # Complexify: a -> (a - b) + b
    (a, (a / b) * b),         # Complexify: a -> (a / b) * b
    (a, a / (a / a)),         # Complexify: a -> a / (a / a)
    (a, (a + b) - (b - c) - c),  # Complexify: a -> (a + b) - (b - c) - c
    (a, (a * b) / (b / c) * c),  # Complexify: a -> (a * b) / (b / c) * c
    (a, ((a * b) + (c * d)) - (c * d)),  # Complexify: a -> ((a * b) + (c * d)) - (c * d)
    (a, (a + (b - c) * c / c)),  # Complexify: a -> a + (b - c) * c / c
    (a, (a + (b / c) * c - b)),  # Complexify: a -> a + (b / c) * c - b
]

def apply_exploration_rule(expr, rule):
    pattern, replacement = rule
    new_expr = expr.replace(pattern, replacement)
    if new_expr != expr:  # Ensure the rule matched and applied
        return new_expr
    return None

def random_walk_with_proof(expr, rules, steps=10):
    proof_steps = [(expr, "Initial expression")]
    for _ in range(steps):
        rule = random.choice(rules)
        new_expr = apply_exploration_rule(expr, rule)
        if new_expr:
            expr = new_expr
            proof_steps.append((expr, f"Applied rule: {rule[0]} -> {rule[1]}"))
    return expr, proof_steps

# Initial expression
expr = x * (y + z)

# Perform random walk with proof steps
new_expr, proof_steps = random_walk_with_proof(expr, exploration_rules, steps=10)

print(f'Original expression: {expr}')
print(f'Rewritten expression: {new_expr}')
print('\nProof steps:')
for step in proof_steps:
    print(f'{step[1]}: {step[0]}')


Original expression: x*(y + z)
Rewritten expression: x*(y + z)

Proof steps:
Initial expression: x*(y + z)


In [ ]:
from sympy import Function
z = Function('z')
#y(x).diff(x)


y = z(x).diff(x)*(sin(cos(x**2))/x)**z(x)
dydx=y.diff(x)

dydx**dydx.diff(x)


u, v = symbols('u v')
w = Function('w')
w(u,v).diff(u,v).diff(u)

In [96]:
import sympy as sp
import random

def explore_expression(expr, depth=0):
    indent = "  " * depth
    print(f"{indent}{expr.__class__.__name__}: {expr}")
    
    if isinstance(expr, sp.Atom):
        return
    
    for arg in expr.args:
        explore_expression(arg, depth + 1)

def apply_rule(expr, rule):
    if callable(rule):
        return rule(expr), True
    else:
        pat, rep = rule
        return expr.replace(pat, rep, simultaneous=False, map=True)

def rewrite(expr, rules, steps=5, max_tries=1000):
    print("Initial expression structure:")
    explore_expression(expr)
    print("\n")

    for step in range(steps):
        print(f"Step {step + 1}:")
        subexprs = list(sp.preorder_traversal(expr))
        
        for _ in range(max_tries):
            subexpr = random.choice(subexprs)
            rule = random.choice(rules)
            
            print(f"Attempting to apply rule {rule} to subexpression: {subexpr}")
            
            if callable(rule):
                new_subexpr = rule(subexpr)
                changed = new_subexpr != subexpr
            else:
                new_subexpr, changed = apply_rule(subexpr, rule)
            
            if changed:
                expr = expr.subs(subexpr, new_subexpr)
                print(f"Rule applied successfully. New expression:")
                explore_expression(expr)
                print("\n")
                break
        else:
            print("Max tries reached without applying a rule.")
        
        print("-" * 50)
    
    print("Final expression structure:")
    explore_expression(expr)
    
    return expr

# Example usage
x, y, z = sp.symbols('x y z')
a, b = sp.Wild('a'), sp.Wild('b')

rules = [
    (sp.Add(a, -a, evaluate=False), 0),
    (a, sp.Add(a, x, -x, evaluate=False), 0),
    (a, sp.Add(a, sp.Integer(0), evaluate=False)),
    (a, sp.Mul(a, sp.Integer(1), evaluate=False)),
    (sp.Integer(0), sp.Mul(a, 0, evaluate=False)),
    (sp.Add(a, sp.Mul(-1, 0, evaluate=False), evaluate=False), a),
    (2 * a, sp.Add(a, a, evaluate=False)),
    (1, sp.sin(x) ** 2 + sp.cos(x) ** 2),
    (1, sp.sin(y) ** 2 + sp.cos(y) ** 2),
    (1, sp.sin(z) ** 2 + sp.cos(z) ** 2),
]

expr = 3 + x
result = rewrite(expr, rules, 5, 100)
print(f"Final expression: {result}")

Initial expression structure:
Add: x + 3
  Integer: 3
  Symbol: x


Step 1:
Attempting to apply rule (a_, -x + x + a_, 0) to subexpression: 3


ValueError: too many values to unpack (expected 2)